In [ ]:
import pandas
import zipfile
import neurom
import os
import numpy

from ipywidgets import widgets, interact
from matplotlib import pyplot as plt

import neurom.apps
import neurom.check
import neurom.check.morphology_checks
import neurom.features
import neurom.geom
import neurom.stats

# Fill in here the name of the morphology to analyze, as downloaded from the platform.
fn_morphology = "download.zip"

# Got this to work. But it reads the whole file at once into memory.
fn_metadata = "metadata.json"

try:
    f_zip = zipfile.ZipFile(fn_morphology)
except zipfile.BadZipFile as e:
    print(f"Zip archive appears to be malformed: {e}")
    raise

zip_contents = list(f_zip.namelist())
zip_contents = [_x for _x in zip_contents if os.path.splitext(_x)[1] != ".h5"]

def morphology_from_zip(z_obj, fn):
    extension = os.path.split(fn)[1][1:]
    with z_obj.open(fn, "r") as fid:
        m_str = fid.read().decode("UTF-8")
    try:
        m = neurom.load_morphology(m_str, reader=extension)
    except Exception as e:
        print("Trouble with " + fn + ": " + str(e))
        return None
    return m


extension_wdgt = widgets.Dropdown(options=zip_contents, description="Choose format to load")
display(extension_wdgt)


Dropdown(description='Choose format to load', options=('/C060114A5.asc', '/C060114A5.swc'), value='/C060114A5.…

In [94]:
morph = morphology_from_zip(f_zip, extension_wdgt.value)
_str = """Loaded morphology with {0} neurites of a total length of {1} um."""
display(_str.format(
    neurom.features.get("number_of_neurites", morph),
    numpy.sum(neurom.features.get("total_length_per_neurite", morph))
))

'Loaded morphology with 11 neurites of a total length of 21728.72217941284 um.'

We calculate for each segment of each neurite its path distance from the soma and its length.

The result is put into a handy DataFrame. For the calculations we use the neurom package.

In [95]:
res = pandas.concat([
    pandas.DataFrame({"lengths": neurom.features.neurite.segment_lengths(_nrt),
                    "pd": neurom.features.neurite.segment_path_lengths(_nrt),
                    "index": i, "type": _nrt.type
                    })
    for i, _nrt in enumerate(morph.neurites)
], axis=0)

display(res.head())

,lengths,pd,index,type
0,0.000000,0.000000,0,NeuriteType.axon
1,2.015146,2.015146,0,NeuriteType.axon
2,1.660789,3.675935,0,NeuriteType.axon
3,1.565399,5.241334,0,NeuriteType.axon
4,2.077421,7.318755,0,NeuriteType.axon


In [96]:
def calculate_results(vld_types, vld_range, show=True):
    msg = "You selected to measure the length of {0}\nbetween {1} and {2} um from the soma."
    if show:
        print(msg.format(vld_types, vld_range[0], vld_range[1]))

    fltr = (res["type"].isin(list(vld_types))) &\
    (res["pd"] >= vld_range[0]) & (res["pd"] < vld_range[1])
    ttl_len = res.loc[fltr]["lengths"].sum()
    if show:
        print("\nThe result in um is shown below:")
    return ttl_len


sel_types = widgets.SelectMultiple(options=res["type"].drop_duplicates(),
                                    index=[],
                                    description="Neurite types")
sel_range = widgets.IntRangeSlider(min=0, max=2000, step=1, description="Select range")
interact(calculate_results, vld_types=sel_types, vld_range=sel_range)

interactive(children=(SelectMultiple(description='Neurite types', options=(<NeuriteType.axon: 2>, <NeuriteType…

<function __main__.calculate_results(vld_types, vld_range, show=True)>

While the above is nice for interactive exploration, the below assigns results to variables that can be used for further analyses.

In [93]:
cutoff_proximal = (0, 100)
cutoff_distal = (100, 250)
used_neurite_types = [neurom.NeuriteType.basal_dendrite, neurom.NeuriteType.apical_dendrite]
tgt_total_count = 250

len_proximal = calculate_results(used_neurite_types, cutoff_proximal, show=False)
len_distal = calculate_results(used_neurite_types, cutoff_distal, show=False)

print("The ratio of proximal over distal dendrite length is {0}".format(
    len_proximal / len_distal
))

req_for_tgt_prox = tgt_total_count / len_proximal
req_for_tgt_dist = tgt_total_count / len_distal

print("""
{0} synapses require a mean length density of {1} on proximal dendrites.
{0} synapses require a mean length density of {2} on distal dendrites.
""".format(tgt_total_count, req_for_tgt_prox, req_for_tgt_dist))


The ratio of proximal over distal dendrite length is 1.3316047607143808

250 synapses require a mean length density of 0.07608212040855789 on proximal dendrites.
250 synapses require a mean length density of 0.10131131374128044 on distal dendrites.

